In [ ]:
%load_ext autoreload
%autoreload 2
import torch as th
from Datasets.TinyShakespeare.prepare import prepare_to_tokenize
from Embeddings.token_embeddings import encode, decode
from TransformerModel.transformer import TransformerDecoder, TransformerTrainer
import json
import numpy as np
from itertools import chain
from Embeddings.constants import EmbedConstants
import random
import wandb
import pickle

In [ ]:
# text = prepare_to_tokenize("/home/skyr/PycharmProjects/TransformerFromScratch/Datasets/Inheritance/big_eragon.txt")
with open("/home/skyr/PycharmProjects/TransformerFromScratch/Datasets/Inheritance/vocab_big.json", "r") as file:
    vocab = json.load(file)

Tiny Shakespeare

In [ ]:
# text = prepare_to_tokenize("/home/skyr/PycharmProjects/TransformerFromScratch/Datasets/TinyShakespeare/input.txt")
# with open("/home/skyr/PycharmProjects/TransformerFromScratch/Datasets/TinyShakespeare/vocab.json", "r") as file:
#     vocab = json.load(file)

In [ ]:
# text = encode(text, vocab)

Save encoded

In [ ]:
# with open("/home/skyr/PycharmProjects/TransformerFromScratch/Datasets/Inheritance/encoded_big.pkl", "wb") as file:
#     pickle.dump(text, file)

Load from pickle

In [ ]:
with open("/home/skyr/PycharmProjects/TransformerFromScratch/Datasets/Inheritance/encoded_big.pkl", "rb") as file:
    text = pickle.load(file)

In [ ]:
def nearest_lower_divisible(number, divisor):
    return number - (number % divisor)


In [ ]:
import math
def nearest_upper_divisible(number,divisor):
    return number + abs(number - divisor * math.ceil(number / divisor))

In [ ]:
num_heads = 8
embedd_dim = nearest_lower_divisible(450,num_heads)
dropout_p = 0.1
feed_forward_size = 1024
num_layers = 8
vocab_size = len(vocab)
# vocab_size = 50257
test_lr = 7e-5
# test_lr = 0.00173770980841192
block_size = 64
# weight_decay = 0.0168053283371547
weight_decay = 1e-5
# weight_decay = 0.007548162081615722
epochs = 20_000
batch_size = 64
eval_every = 200
# {'num_heads': 8, 'factor': 4, 'dropout_p': 0.25068545758071636, 'feed_forward_size': 731, 'num_layers': 6, 'block_size': 97, 'lr': 0.00173770980841192, 'weight_decay': 0.0168053283371547}

In [ ]:
embedd_dim

In [ ]:
wandb.init(
    project="transformer_from_scratch_inheritance",
    config={
        "embedd_dim": embedd_dim,
        "num_heads": num_heads,
        "dropout_p": dropout_p,
        "feed_forward_size": feed_forward_size,
        "num_layers": num_layers,
        "vocab_size": vocab_size,
        "test_lr": test_lr,
        "block_size": block_size,
        "weight_decay": weight_decay,
        "epochs": epochs,
        "batch_size": batch_size
    }

)

In [ ]:
# table = wandb.Table(columns=["predicted_text", "real_text"])
table = None

In [ ]:
transformer = TransformerDecoder(embedd_dim, num_heads, dropout_p, feed_forward_size, num_layers, vocab_size,
                                     block_size)
trainer = TransformerTrainer(transformer, th.optim.AdamW, th.nn.CrossEntropyLoss, test_lr, block_size, weight_decay,vocab, table)

Load state dict

In [ ]:
# data = th.load("/home/skyr/PycharmProjects/TransformerFromScratch/model2.pth")
# transformer.load_state_dict(data["model_state_dict"])
# trainer.optimizer.load_state_dict(data["optimizer_state_dict"])
# trainer.optimizer 

In [ ]:
data = list(chain.from_iterable(text))

In [ ]:
test = data[:int(len(data) * 0.1)]
val = data[int(len(data) * 0.1):int(len(data) * 0.2)]
train = data[int(len(data) * 0.2):]
train = th.tensor(train).long()
# val = th.tensor(val).long()
test = th.tensor(test).long()

In [ ]:
# print(len(train))
trainer.train(train, test, epochs, batch_size,eval_every=eval_every)

In [ ]:
th.save({
        "model_state_dict": transformer.state_dict(),
        "optimizer_state_dict": trainer.optimizer.state_dict(),
    }, "model3.pth")
# trainer.draw_losses()

In [ ]:
input_ = encode(["Eragon"],vocab)
input_ = th.tensor(input_).long().cuda().reshape(1, -1)
outputs = decode(transformer.generate(input_, 108, block_size), vocab)
print(''.join(outputs))

In [ ]:
# wandb.finish()